In [96]:
import jax
import jax.numpy as jnp
from jax import jit
import jax.lax as lax
from jax import make_jaxpr

In [97]:
def func(x, y):
    #return x if x.sum() < y.sum() else y # leads to TracerBoolConversionError
    return jnp.where(x.sum() < y.sum(), x, y) # workaround: use whre

In [98]:
x, y = jnp.array([1, 2, 3]), jnp.array([0])
jit(func)(x, y)
#make_jaxpr(func)(x, y)

Array([0, 0, 0], dtype=int32)

In [99]:
def func(i, x):
  return x + i

def simulate_finite_time(x):
    trajectory = []
    for i in range(10):
        x = func(i, x)
        trajectory.append(x)
    return jnp.array(trajectory)

In [100]:
#jit(simulate_finite_time)(-10)
#make_jaxpr(simulate_finite_time)(0)
lax.fori_loop(lower=0, upper=10, body_fun=func, init_val=-10)

Array(35, dtype=int32, weak_type=True)

In [101]:
def simulate_fht2(x):
    trajectory = []
    i = 0
    while x != 5:
        x = x + i
        trajectory.append(x)
        i = i + 1
    return jnp.array(trajectory)

In [106]:
simulate_fht2(-5)
#jit(simulate_fht2)(2)

Array([-5, -4, -2,  1,  5], dtype=int32)

In [108]:
def body_fun(carry):
  i, x = carry
  return i + 1, func(i, x)
    
def cond_fun(carry):
  i, x = carry
  break_condition = (x == 5)
  return ~break_condition & (i < 10)

In [109]:
lax.while_loop(cond_fun, body_fun, init_val=(0, -5))

(Array(5, dtype=int32, weak_type=True), Array(5, dtype=int32, weak_type=True))